In [1]:
import numpy as np
import pandas as pd

import sys
import tensorflow as tf

2024-02-10 22:47:17.344502: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-10 22:47:17.344541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-10 22:47:17.346175: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-10 22:47:17.354823: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-10 22:47:18.546040: W tensorflow/compiler/tf2

In [2]:
sys.path.append('..')
sys.path.append('../..')

In [3]:
from src.Environment.environment import MyModelSelectionEnv
from src.utils import train_test_anomaly

In [4]:
import tf_agents.bandits.agents as bandit_agents
from tf_agents.specs import array_spec, tensor_spec
from tf_agents.trajectories import time_step as ts
from tf_agents.environments import tf_py_environment
from tf_agents.trajectories import trajectory
from tf_agents.trajectories import policy_step
from tf_agents.networks import q_network

In [5]:
file_path = '../datasets/Dodgers/101-freeway-traffic.test.out'

df = pd.read_csv(file_path)

In [6]:
column_names = {'-1': 'value', '1': 'anomaly'}
df.rename(columns=column_names, inplace=True)

In [7]:
_, test_data = train_test_anomaly(df)

In [8]:
list_threshold = [-0.03, +5]

In [9]:
list_gtruth = test_data['anomaly']

In [10]:
env = MyModelSelectionEnv(test_data, list_thresholds=list_threshold, list_gtruth=list_gtruth)
environment = tf_py_environment.TFPyEnvironment(env)

In [11]:
action_spec = tensor_spec.BoundedTensorSpec(shape= () ,dtype=np.int32, minimum=0, maximum = 1, name='Select Models')
observation_spec = tensor_spec.BoundedTensorSpec(shape=(1, 159), dtype=np.float64, minimum=-9999999, maximum=9999999, name='observation')
time_step_spec = ts.time_step_spec(observation_spec)

EPSILON = 0.05
LAYERS = (50, 50, 50)
LR = 0.005

network = q_network.QNetwork(
          input_tensor_spec=observation_spec,
          action_spec=action_spec,
          fc_layer_params=LAYERS
      )

In [12]:

eps_agent = bandit_agents.neural_epsilon_greedy_agent.NeuralEpsilonGreedyAgent(action_spec=action_spec, time_step_spec=time_step_spec, reward_network=network ,optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=LR),
        epsilon=EPSILON,
        emit_policy_info='predicted_rewards_mean',
        info_fields_to_inherit_from_greedy=['predicted_rewards_mean'])

In [13]:
def trajectory_for_bandit(initial_step, action_step, final_step):
  return trajectory.Trajectory(observation=initial_step.observation,
                               action=action_step.action,
                               policy_info=action_step.info,
                               reward=final_step.reward,
                               discount=final_step.discount,
                               step_type=initial_step.step_type,
                               next_step_type=final_step.step_type)


In [14]:
step = environment.reset()

for i in range(10):

    action_step = eps_agent.collect_policy.action(step)
    print(action_step.action)
    next_step = environment.step(action_step.action)
    experience = trajectory_for_bandit(step, action_step, next_step) # Error in this line. Figure out the trajectory for bandit data type. 
    print(experience)
    eps_agent.train(experience=experience)
    step = next_step
    print(f'step {i} complete')
    

*** Feature extraction started ***



*** Feature extraction finished ***
tf.Tensor([1], shape=(1,), dtype=int32)
*** Feature extraction started ***



*** Feature extraction finished ***
Trajectory(
{'step_type': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>,
 'observation': <tf.Tensor: shape=(1, 1, 159), dtype=float64, numpy=
array([[[ 2.26980000e+04,  7.72500000e+00,  2.26980000e+04,
          4.63224490e+04,  2.79830382e-01,  1.00000000e+01,
          4.00000000e+01, -1.00000000e+00,  2.80000000e+01,
          2.00000000e-02,  4.00000000e-02,  6.00000000e-02,
          8.00000000e-02,  1.00000000e-01,  1.20000000e-01,
          1.40000000e-01,  1.60000000e-01,  1.80000000e-01,
          2.00000000e-01,  7.91601227e-01,  4.06120456e-03,
          1.66111210e+00,  3.34111507e+00,  7.36808086e-02,
          1.29475394e+00,  1.95294960e+00,  1.86123572e+00,
          2.19119116e+00,  2.16909364e+00,  1.59364940e+00,
          2.79607727e+00,  1.45212482e+01,  1.15778395e+01,
          4.01796542e+00,  2.99860121e+00,  7.39479772e+00,
          4.90333133e+00,  1.33782826e+01,  3.41726431e-02,
          1.1181036

/home/rojan/Desktop/projects/MAB-Model-Selection/MAB-Based-Model-Selection/venv/lib/python3.9/site-packages/tf_agents/utils/common.py:1539: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return state is not None and state is not () and state is not []
/home/rojan/Desktop/projects/MAB-Model-Selection/MAB-Based-Model-Selection/venv/lib/python3.9/site-packages/tf_agents/utils/common.py:1539: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return state is not None and state is not () and state is not []


ValueError: All of the Tensors in `value` must have two outer dimensions. Specifically, tensors must have shape `[B, T] + spec.shape`.
Full shapes of value tensors:
  Trajectory(
{'step_type': TensorShape([1]),
 'observation': TensorShape([1, 1, 159]),
 'action': TensorShape([1]),
 'policy_info': PolicyInfo(log_probability=(), predicted_rewards_mean=TensorShape([1, 2]), multiobjective_scalarized_predicted_rewards_mean=(), predicted_rewards_optimistic=(), predicted_rewards_sampled=(), bandit_policy_type=()),
 'next_step_type': TensorShape([1]),
 'reward': TensorShape([1]),
 'discount': TensorShape([1])}).
Expected shapes (excluding the two outer dimensions):
  Trajectory(
{'step_type': TensorShape([]),
 'observation': TensorShape([1, 159]),
 'action': TensorShape([]),
 'policy_info': PolicyInfo(log_probability=(), predicted_rewards_mean=TensorShape([2]), multiobjective_scalarized_predicted_rewards_mean=(), predicted_rewards_optimistic=(), predicted_rewards_sampled=(), bandit_policy_type

In [ ]:
action_spec = tensor_spec.BoundedTensorSpec(shape= () ,dtype=np.int32, minimum=0, maximum = 1, name='Select Models')
observation_spec = tensor_spec.BoundedTensorSpec(shape=(1, 318), dtype=np.float64, minimum=-9999999, maximum=9999999, name='observation')
time_step_spec = ts.time_step_spec(observation_spec)

linucb_agent = bandit_agents.lin_ucb_agent.LinearUCBAgent(action_spec=action_spec, time_step_spec=time_step_spec, tikhonov_weight=0.001, alpha=10)

In [ ]:
step = environment.reset()

for i in range(10):

    action_step = agent._collect_policy.action(step)
    next_step = environment.step(action_step)
    experience = trajectory_for_bandit(step, action_step, next_step)
    print(experience)
    agent.train(experience=experience)
    step = next_step
    